In [12]:
import pandas as pd 
import numpy as np 
from scipy import sparse
from mlxtend.preprocessing import TransactionEncoder
from collections import defaultdict

In [3]:
dfOrder = pd.read_csv('order_products__train.csv')
dfOrder

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [4]:
dfProduct = pd.read_csv('products.csv')
dfProduct

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [ ]:
#Etapa 1, tarea 2
#Identificacion de los N productos mas frecuentes
N = 1000 #Este numero puede variar, solo considerar mientras mas alto, mas es el coste computacional 

top_prod = dfOrder['product_id'].value_counts().nlargest(N).index

In [ ]:
#Filtro para incluir los productos frecuentes
dfOrder_fill = dfOrder[dfOrder['product_id'].isin(top_prod)]

#Creamos matriz binaria
order_matrix = pd.crosstab(dfOrder_fill['order_id'], dfOrder_fill['product_id'],  dropna=False)


In [ ]:
#Conversion binario 1 para presencia, 0 para ausencia
dfBi = (order_matrix > 0).astype(int)
dfBi.head()

product_id,34,45,196,260,311,329,365,432,581,651,...,49175,49191,49235,49247,49383,49520,49533,49605,49610,49683
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
36,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Esto es para verificar si me esta arrojando los datos correctos(fin informativo)
print(f"Forma del DataFrame: {dfBi.shape}")
print(f"Valores únicos: {np.unique(dfBi.values)}")
print(f"Número de True: {(dfBi == True).values.sum()}")
print(f"Número de False: {(dfBi == False).values.sum()}")
print(f"Porcentaje de True: {(dfBi == True).values.sum() / dfBi.size * 100:.2f}%")

Forma del DataFrame: (120218, 1000)
Valores únicos: [0 1]
Número de True: 728595
Número de False: 119489405
Porcentaje de True: 0.61%


In [ ]:
#Etapa 1, tarea 2 continuacion
#Umbral de soporte minimo (es de ejemplo modificable si es necesario)
min_support = 0.01

#Calculo de soporte de cada item
item_support = dfBi.sum() / len(dfBi) #Esto es la frecuencia de aparicion del producto en el total de transacciones

#aca volvemos a filtrar para ver si cumplem el soporte minimo colocado
frequent_item = item_support[item_support >= min_support]

In [ ]:
#ordene por soporte de forma descendente
frequent_item = frequent_item.sort_values(ascending=False)

#Visualizacion de respuesta y verificacion de que el codigo funciona
print(f"1-itemsets frecuentes encontrados: {len(frequent_item)}")
print("\nTop 10 productos más frecuentes:")
print(frequent_item.head(10))

1-itemsets frecuentes encontrados: 118

Top 10 productos más frecuentes:
product_id
24852    0.155767
13176    0.128766
21137    0.090619
21903    0.081385
47626    0.067669
47766    0.061630
47209    0.060665
16797    0.054019
26209    0.050184
27966    0.046133
dtype: float64


In [ ]:
# Crear un DataFrame con los 1-itemsets frecuentes para uso posterior
frequentDf = pd.DataFrame({'itemset': [[product] for product in frequent_item.index], 'support': frequent_item.values})

print("DataFrame de 1-itemsets frecuentes:")
print(frequent_item)

#Guardado los indices de los items frecuentes (esto podemos usarlo para generar 2-itemsets)
frequent_items = frequent_item.index.tolist()


DataFrame de 1-itemsets frecuentes:
product_id
24852    0.155767
13176    0.128766
21137    0.090619
21903    0.081385
47626    0.067669
           ...   
41844    0.010132
260      0.010107
24489    0.010090
41787    0.010040
38689    0.010032
Length: 118, dtype: float64


In [ ]:
#Etapa 1, tarea 3
def estructuracion(dfOrder, top_n= 1000):
    #Obtengo lso prodcutos mas frecuentes
    top_prod= dfOrder['product_id'].valuee_counts().nlargest(top_n).index
    #filtro para incluir dichos productos
    dfOrder_fill = dfOrder[dfOrder['product_id'].isin(top_prod)]
    #Aqui creo el dicci
    transactions_dict = defaultdict(set)
    #Hago una funcion para agrupar los productos por orden
    for _, row in dfOrder_fill.iterrows():
        transactions_dict[row['order_id']].add(row['product_id'])
    
    return dict(transactions_dict)